## Imports des modules

In [1]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.misc import imread
from sklearn.externals import joblib

import random as rand
import numpy as np 
import cv2
import glob
import time

import matplotlib.pyplot as plt
%matplotlib inline

from featuresourcer import FeatureSourcer

import warnings
warnings.filterwarnings('ignore')

## Initialisation paramètres

In [2]:
#sourcer_params = {
#  'color_model': 'yuv',                # hls, hsv, yuv, ycrcb
#  'bounding_box_size': 128,             #
#  'number_of_orientations': 11,        # 6 - 12
#  'pixels_per_cell': 16,               # 8, 16
#  'cells_per_block': 2,                # 1, 2
#  'do_transform_sqrt': True
#}

sourcer_params = {
    'bounding_box_size': 96,
    'cells_per_block': 2,
    'color_model': 'hsv',
    'number_of_orientations': 6,
    'pixels_per_cell': 8
}

img_size = sourcer_params['bounding_box_size']

sourcer = FeatureSourcer(sourcer_params)

## Import dataset

In [3]:
print("Loading images to memory...")
t_start = time.time()

insect_imgs, non_insect_imgs = [], []
insect_paths = glob.glob('datasets/cropped/insect/**/*.png')
non_insect_paths = glob.glob('datasets/cropped/non_insect/*.png')

for path in insect_paths: insect_imgs.append(cv2.resize(cv2.imread(path),(img_size, img_size)))
for path in non_insect_paths: non_insect_imgs.append(cv2.resize(cv2.imread(path),(img_size, img_size)))
    
insect_imgs, non_insect_imgs = np.asarray(insect_imgs), np.asarray(non_insect_imgs)
total_insects, total_non_insects = insect_imgs.shape[0], non_insect_imgs.shape[0]

print("... Done")
print("Time Taken:", np.round(time.time() - t_start, 2), "s")
print("Insect images shape: ", insect_imgs.shape)
print("Non-Insect images shape: ", non_insect_imgs.shape)

Loading images to memory...
... Done
Time Taken: 2.1 s
Insect images shape:  (85, 96, 96, 3)
Non-Insect images shape:  (87, 96, 96, 3)


## Extraction features

In [4]:
print("Extracting features... This might take a while...")
t_start = time.time()

insect_features, non_insect_features = [], []

print("Insect...")
for img in insect_imgs:
    insect_features.append(sourcer.features(img))
    print('█', end = '')

print()
print("Non-Insect...")
for img in non_insect_imgs:
    non_insect_features.append(sourcer.features(img))
    print('█', end = '')
    
insect_features = np.asarray(insect_features)
non_insect_features = np.asarray(non_insect_features)

print()
print("...Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print("Insect features shape: ", insect_features.shape)
print("Non-Insect features shape: ", non_insect_features.shape)

Extracting features... This might take a while...
Insect...
(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96, 96, 3)
█(96,

## Normalisation des features

In [5]:
print("Scaling features...")
t_start = time.time()

print(insect_features.shape)
print(non_insect_features.shape)

unscaled_x = np.vstack((insect_features, non_insect_features)).astype(np.float64)
scaler = StandardScaler().fit(unscaled_x)
x = scaler.transform(unscaled_x)
y = np.hstack((np.ones(total_insects), np.zeros(total_non_insects)))

print("...Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print("x shape: ", x.shape, " y shape: ", y.shape)

Scaling features...
(85, 8712)
(87, 8712)
...Done
Time Taken: 0.04
x shape:  (172, 8712)  y shape:  (172,)


## Séparation en dataset entrainement et test

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=41)

## Entrainement modèle

In [7]:
print("Training classifier...")
t_start = time.time()

svc = LinearSVC()
svc.fit(x_train, y_train)
accuracy = svc.score(x_test, y_test)

print("...Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print("Accuracy: ", np.round(accuracy, 4))

Training classifier...
...Done
Time Taken: 0.08
Accuracy:  0.9429


## Sauvegarde modèle

In [8]:
print ("Saving models...")

joblib.dump(svc, 'models/svc2.pkl')
joblib.dump(scaler, 'models/scaler2.pkl')

print("...Done")

Saving models...
...Done


In [9]:
print ("Loading models...")

svc = joblib.load('models/svc2.pkl')
scaler = joblib.load('models/scaler2.pkl')

print("...Done")

Loading models...
...Done


## Test manuel du modèle

In [10]:
#sourcer = FeatureSourcer(sourcer_params)
print(total_non_insects)
f1 = sourcer.features(non_insect_imgs[rand.randint(0, total_non_insects - 1)])
f2 = sourcer.features(insect_imgs[rand.randint(0, total_insects - 1)])
f3 = sourcer.features(insect_imgs[rand.randint(0, total_insects - 1)])
f4 = sourcer.features(non_insect_imgs[rand.randint(0, total_non_insects - 1)])

f = scaler.transform([f1, f2, f3, f4])
print(svc.predict(f))

87
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
[0. 1. 1. 0.]
